<a href="https://colab.research.google.com/github/nattycep/FIL_Geosensing/blob/main/DataVis_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import, Visualization, and Troubleshooting with Python

**Expected study time :** 45 min (depending on your background)

**Learning outcomes :**

*Adapt an existing Python script in google colab in order to:*
- Load data from your geosensor
- Import data into Python
- Visualize the data
- Refine and export figures for presentation
- Perform basic statistics to support a scientific finding 

**External References**
- the Scipy.stat module https://docs.scipy.org/doc/scipy/reference/stats.html


# 1.   Load Packages



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

# 2.   Load Data 


# 3.   Make Figures: Preliminary, Refine, Export




# 4.  Perform Statistical Tests 
